In [1]:
import subprocess
import threading
import time
import requests

# Function to launch vllm in background
def launch_vllm_server():
    command = [
        "vllm",
        "serve",
        "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
        "--trust-remote-code",
        "--port", "8000",
        "--dtype", "half",
        "--max-model-len", "16384",
        "--enable-chunked-prefill", "true"
    ]

    def run():
        subprocess.run(command)

    thread = threading.Thread(target=run, daemon=True)
    thread.start()
    print("🚀 vLLM server is launching...")

# Call this once in the notebook
launch_vllm_server()


🚀 vLLM server is launching...


In [7]:
!pip install fastapi nest-asyncio pyngrok uvicorn

In [2]:
import requests

def ask_model(question, model="facebook/opt-350m"):
    url = "http://localhost:8000/v1/chat/completions"
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": question}],
        "max_tokens": 100
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        print("❌ Error communicating with model:", e)
        return None


INFO 03-22 05:32:21 [__init__.py:256] Automatically detected platform cpu.
INFO 03-22 05:32:21 [api_server.py:977] vLLM API server version 0.8.1
INFO 03-22 05:32:21 [api_server.py:978] args: Namespace(subparser='serve', model_tag='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', config='', host=None, port=8000, uvicorn_log_level='info', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key=None, lora_modules=None, prompt_adapters=None, chat_template=None, chat_template_content_format='auto', response_role='assistant', ssl_keyfile=None, ssl_certfile=None, ssl_ca_certs=None, enable_ssl_refresh=False, ssl_cert_reqs=0, root_path=None, middleware=[], return_tokens_as_token_ids=False, disable_frontend_multiprocessing=False, enable_request_id_headers=False, enable_auto_tool_choice=False, tool_call_parser=None, tool_parser_plugin='', model='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', task='auto', tokenizer=None, hf_config_path=None, skip_tokenizer_ini

Process SpawnProcess-1:
Traceback (most recent call last):
  File "/Users/chivo/.pyenv/versions/3.10.12/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/chivo/.pyenv/versions/3.10.12/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/chivo/.pyenv/versions/3.10.12/lib/python3.10/site-packages/vllm/engine/multiprocessing/engine.py", line 450, in run_mp_engine
    raise e
  File "/Users/chivo/.pyenv/versions/3.10.12/lib/python3.10/site-packages/vllm/engine/multiprocessing/engine.py", line 436, in run_mp_engine
    engine = MQLLMEngine.from_vllm_config(
  File "/Users/chivo/.pyenv/versions/3.10.12/lib/python3.10/site-packages/vllm/engine/multiprocessing/engine.py", line 128, in from_vllm_config
    return cls(
  File "/Users/chivo/.pyenv/versions/3.10.12/lib/python3.10/site-packages/vllm/engine/multiprocessing/engine.py", line 82, in __init__
    self.engine = LLMEngine(*

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

inputs = tokenizer("Tell me a sarcastic caption about frogs.", return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Tell me a sarcastic caption about frogs. I want to use it in a social media post. It should be humorous but not offensive. I need to include the frog's life stages: egg, tadpole, tad, frog. Also, I should mention the frog's lifespan and why frogs


In [5]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
outputs = model.generate(**inputs, max_new_tokens=150)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Tell me a sarcastic caption about frogs. I want to use it in a social media post. It should be humorous but not offensive. I need to include the frog's life stages: egg, tadpole, tad, frog. Also, I should mention the frog's lifespan and why frogs


In [12]:
!pip show deepseek_vl

Name: deepseek_vl
Version: 1.0.0
Summary: DeepSeek-VL
Home-page: https://github.com/deepseek-ai/DeepSeek-VL
Author: DeepSeek-AI
Author-email: 
License: MIT License
        
        Copyright (c) 2023 DeepSeek
        
        Permission is hereby granted, free of charge, to any person obtaining a copy
        of this software and associated documentation files (the "Software"), to deal
        in the Software without restriction, including without limitation the rights
        to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
        copies of the Software, and to permit persons to whom the Software is
        furnished to do so, subject to the following conditions:
        
        The above copyright notice and this permission notice shall be included in all
        copies or substantial portions of the Software.
        
        THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
        IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANT

In [13]:
import sys
sys.path.append("/Users/chivo/DeepSeek-VL")


In [22]:
import os
import pandas as pd
import torch
from transformers import AutoModelForCausalLM
from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM
from deepseek_vl.utils.io import load_pil_images

# Model path
model_path = "deepseek-ai/deepseek-vl-7b-chat"

# Load model and processor
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

# Load the model and move to float16 or float32 as needed
vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.half().to(device).eval()  # or use .float() for float32


# Paths
image_folder = "contest_images"
existing_csv_path = "/Users/chivo/Downloads/data_studio/project4/new_yorker_contest/ai_captions_deepseek.csv"

# Load existing captions
if os.path.exists(existing_csv_path):
    existing_df = pd.read_csv(existing_csv_path)
    already_captioned_ids = set(existing_df["contest"].astype(str))
    print(f"📄 Found existing captions for {len(already_captioned_ids)} contests.")
else:
    existing_df = None
    already_captioned_ids = set()
    print("📄 No existing captions found. Starting fresh.")

output = []

# Loop through images
for image_name in os.listdir(image_folder):
    if not image_name.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    contest_id = os.path.splitext(image_name)[0]
    if contest_id in already_captioned_ids:
        print(f"⏩ Skipping {contest_id} (already captioned)")
        continue

    image_path = os.path.join(image_folder, image_name)
    prompt = "Write a clever one-line caption for a New Yorker-style cartoon."

    # Set up conversation
    conversation = [
        {
            "role": "User",
            "content": f"<image_placeholder>{prompt}",
            "images": [image_path]
        },
        {"role": "Assistant", "content": ""}
    ]

    try:
        # Load and process image
        pil_images = load_pil_images(conversation)
        prepare_inputs = vl_chat_processor(
            conversations=conversation,
            images=pil_images,
            force_batchify=True
        ).to(device)

        inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)
        inputs_embeds = {
            k: v.to(dtype=torch.float16) if isinstance(v, torch.Tensor) else v
            for k, v in inputs_embeds.items()
        }



        # Generate caption
        outputs = vl_gpt.language_model.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=prepare_inputs.attention_mask,
            pad_token_id=tokenizer.eos_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=64,
            do_sample=True,
            temperature=0.9,
            top_p=0.9,
            repetition_penalty=1.3
        )

        caption = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True).replace(prompt, "").strip()
        print(f"✅ [{contest_id}] Caption: {caption}")

        output.append({
            "contest": contest_id,
            "model": "DeepSeek-VL-7B",
            "caption": caption
        })

    except Exception as e:
        print(f"❌ Error processing {contest_id}: {e}")
        continue

# Save to CSV
if output:
    new_df = pd.DataFrame(output)
    if existing_df is not None:
        full_df = pd.concat([existing_df, new_df], ignore_index=True)
    else:
        full_df = new_df
    full_df.to_csv(existing_csv_path, index=False)
    print(f"💾 Saved {len(new_df)} new captions to: {existing_csv_path}")
else:
    print("✅ All captions already exist. Nothing new to save.")


Some kwargs in processor config are unused and will not have any effect: mask_prompt, image_tag, add_special_token, ignore_id, sft_format, num_image_tokens. 


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

📄 No existing captions found. Starting fresh.
❌ Error processing 807_Dashboard: Input type (c10::BFloat16) and bias type (c10::Half) should be the same
❌ Error processing 689_Dashboard: Input type (c10::BFloat16) and bias type (c10::Half) should be the same
❌ Error processing 538_Dashboard: Input type (c10::BFloat16) and bias type (c10::Half) should be the same
❌ Error processing 634_Dashboard: Input type (c10::BFloat16) and bias type (c10::Half) should be the same
❌ Error processing 764_Dashboard: Input type (c10::BFloat16) and bias type (c10::Half) should be the same
❌ Error processing 585_Dashboard: Input type (c10::BFloat16) and bias type (c10::Half) should be the same
❌ Error processing 623_Dashboard: Input type (c10::BFloat16) and bias type (c10::Half) should be the same
❌ Error processing 773_Dashboard: Input type (c10::BFloat16) and bias type (c10::Half) should be the same
❌ Error processing 592_Dashboard: Input type (c10::BFloat16) and bias type (c10::Half) should be the same


In [2]:
full_df

,contest,model,caption
0,807_Dashboard,DeepSeek (local),_3_Cover_619.\n\nThe original text is as follo...
1,689_Dashboard,DeepSeek (local),_1000_Square_Cartoon\nThe contest is about cre...
2,538_Dashboard,DeepSeek (local),The image in the previous post is [the graph o...
3,634_Dashboard,DeepSeek (local),"_2015.\nThe description says:\n""Find the maxim..."
4,764_Dashboard,DeepSeek (local),_21908_Cartoon. You can use the following word...
...,...,...,...
380,715_Dashboard,DeepSeek (local),".\nAlright, so I need to write a one-liner tha..."
381,645_Dashboard,DeepSeek (local),_3_Cooning.\n\nThe goal is to make the image a...
382,702_Dashboard,DeepSeek (local),_Carrot_Wrap.\nThe contest involves writing an...
383,652_Dashboard,DeepSeek (local),(the new dashboard) is being presented in the ...


In [7]:
import os
import base64
import pandas as pd
import requests
from dotenv import load_dotenv
import time

# Load API key
load_dotenv()
deepseek_key = os.getenv("DEEPSEEK_API_KEY")

headers = {
    "Authorization": f"Bearer {deepseek_key}",
    "Content-Type": "application/json"
}

# Folder where your cartoons live
image_folder = "contest_images"
output = []

def call_deepseek_vision(img_path, contest_id):
    with open(img_path, "rb") as img_file:
        encoded = base64.b64encode(img_file.read()).decode("utf-8")

    print(f"\n📷 Sending image: {img_path}, size = {os.path.getsize(img_path)/1024:.1f} KB")

    body = {
        "model": "deepseek-vl",  # ✅ fixed model name
        "messages": [
            {
                "role": "user",
                "content": "Write a clever, dry, or absurd one-line caption. Think like The New Yorker."
            }
        ],
        "image": f"data:image/jpeg;base64,{encoded}",
        "max_tokens": 100
    }

    try:
        response = requests.post("https://api.deepseek.com/v1/chat/completions", headers=headers, json=body)
        response.raise_for_status()
        caption = response.json()["choices"][0]["message"]["content"].strip()
        print(f"✅ [{contest_id}] Caption: {caption}")
        return caption

    except requests.exceptions.HTTPError as e:
        print(f"❌ [{contest_id}] HTTP Error: {e}")
        print("🔍 DeepSeek response:", response.text)
        return None

# Caption every image
for image_name in os.listdir(image_folder):
    if not image_name.lower().endswith(".jpg"):
        continue

    contest_id = os.path.splitext(image_name)[0]
    image_path = os.path.join(image_folder, image_name)

    caption = call_deepseek_vision(image_path, contest_id)
    if caption:
        output.append({
            "contest": contest_id,
            "model": "DeepSeek-VL",
            "caption": caption
        })

    time.sleep(2)  # polite pause to avoid rate limit

# Save all captions
df = pd.DataFrame(output)
csv_path = "captions_deepseek_vl.csv"
df.to_csv(csv_path, index=False)
print(f"\n💾 Captions saved to {csv_path}")



📷 Sending image: contest_images/807_Dashboard.jpg, size = 1020.4 KB
❌ [807_Dashboard] HTTP Error: 400 Client Error: Bad Request for url: https://api.deepseek.com/v1/chat/completions
🔍 DeepSeek response: {"error":{"message":"Model Not Exist","type":"invalid_request_error","param":null,"code":"invalid_request_error"}}

📷 Sending image: contest_images/689_Dashboard.jpg, size = 396.6 KB


KeyboardInterrupt: 

In [2]:
"model": "deepseek-vl-7b"


SyntaxError: illegal target for annotation (2923736107.py, line 1)

In [5]:
import requests

def ask_model(contest_id, model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"):
    url = "http://localhost:8000/v1/chat/completions"
    headers = {"Content-Type": "application/json"}

    # 📜 Prompt template
    prompt = (
        f"This is a caption contest cartoon from contest ID {contest_id}. "
        "Write a witty, clever, absurd, or dry one-line caption as if it were for The New Yorker. "
        "Do not explain anything. Just output the caption. Keep it under 30 words."
    )

    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 100,
        "temperature": 0.9,
        "top_p": 0.95,
        "repetition_penalty": 1.2
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"].strip()
    except requests.exceptions.RequestException as e:
        print(f"❌ [Error for {contest_id}]:", e)
        return None


In [ ]:
import json

def ask_model(question, model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"):
    url = "http://localhost:8000/v1/chat/completions"
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": question}],
        "max_tokens": 100
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        print("❌ Error communicating with model:", e)
        return None

# Example usage
response = ask_model("Write a funny cartoon caption about two cats arguing over a pizza.")
print("📝 Model response:", response)


In [6]:
caption = ask_model("895_Dashboard")
print(f"📝 Caption for 895_Dashboard:\n{caption}")


❌ [Error for 895_Dashboard]: 404 Client Error: Not Found for url: http://localhost:8000/v1/chat/completions
📝 Caption for 895_Dashboard:
None
